# Energy Consumption Analysis for 15-minute Interval Data

This notebook provides a comprehensive analysis of energy consumption data that is recorded at 15-minute intervals. Our goal is to derive meaningful insights from the data and answer specific questions related to energy usage patterns.

**Key Objectives:**
1. Determine annual energy consumption.
2. Calculate the annual cost based on a given electricity price per kWh.
3. Analyze daily consumption patterns and monthly breakdowns.
4. Study consumption differences across days of the week.
5. Examine intra-day consumption distribution.
6. Identify days with peak consumption at 15-minute intervals.
7. Highlight the top 5 days with the highest instantaneous power usage.
8. Create a histogram to visualize quarterly consumption values relative to the total.
9. Compare solar panel production against consumption.
10. Optimize and understand car charger energy consumption.

## Beginning the Analysis

In this analysis, we examine the energy consumption data from multiple meters. The dataset provides consumption values in 15-minute intervals. Our goal is to preprocess this data and derive valuable insights from various perspectives, such as annual, daily, and intra-day consumption patterns.

### Setting up the Environment

To ensure our analysis runs smoothly, we've taken the following steps:

**Created a Virtual Environment:** This helps to keep our project dependencies separate from other Python projects. By using a virtual environment, we ensure that our project runs consistently across different setups.

**Installed Necessary Libraries:** Our project leverages several Python libraries that aid in data manipulation, visualization, and analysis. These libraries have been automatically installed in the virtual environment to guarantee the reproducibility of our analysis.

**Imported Libraries:** The required libraries for this notebook have been imported upfront to streamline the analysis process. These libraries provide a variety of functions and tools essential for our tasks.

**Importing Custom Utility Functions:** To modularize our analysis and promote code reusability, we've defined custom functions in separate Python files. In this section, we'll import these utility functions to be used throughout our notebook.

**Loaded Configuration:** Constants and configurations, such as file paths and threshold values, are sourced from a .env file. This approach promotes better code management and security


### Activating the Virtual Environment

Once you've set up the project with Poetry, you can activate the virtual environment across any operating system using the following command:

```bash
poetry shell
```

This command will start a new shell session with the virtual environment activated. All the dependencies installed via Poetry will be available in this shell session.

Then you have to install Jupyter Kernel for the virtual environment:

```bash
python -m ipykernel install --user --name=zengrid-analysis-poetry
```

### Importing Libraries



In [ ]:
# ---------------
# IMPORTING LIBRARIES
# ---------------

# Standard Libraries
import os
import calendar
import re
import sys
import logging

# Data Processing and Analysis Libraries
import pandas as pd
import numpy as np

# Visualization Libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Configuration and Environment Libraries
from dotenv import load_dotenv

### Creating the .env file

In [ ]:
# ---------------
# SETTING UP .ENV FILE
# ---------------

sys.path.append('../..')  # Adjust the path to ensure it's pointing to the directory containing environment_setup.py.
import src # Contains custom utility functions for our analysis
from utils import data_utils as du


# Constants for column endings
SOLAR_PANEL_SUFFIX = "/PV"
ELECTRIC_CAR_CHARGER_SUFFIX = "/ECC"
BATTERY_SUFFIX = "/BAT"

# Default values for .env file
SOURCE_FILE_PATH = "YOUR_FILE_PATH_HERE"
EXPORT_FILE_PATH="YOUR_EXPORT_DIR_HERE"
PRICE_PER_KWH = 0.000
THRESHOLD_FOR_OUTLIERS = 1.5

# Check if the user wants to overwrite the existing .env file
force_create = input("Do you want to overwrite the existing .env file if it exists? (yes/no): ").strip().lower() == "yes"

# Create or update the .env file with configuration values
src.create_dotenv_file_cons_analysis(force_create, SOLAR_PANEL_SUFFIX, ELECTRIC_CAR_CHARGER_SUFFIX, BATTERY_SUFFIX, SOURCE_FILE_PATH, EXPORT_FILE_PATH, PRICE_PER_KWH, THRESHOLD_FOR_OUTLIERS)

# TODO: Expand on adding currency. Consider incorporating different currencies and conversion rates?
# TODO: Add RoI, price for solar panels, and other related metrics to provide a more comprehensive financial analysis.

### Configuring variables

In [ ]:
# ---------------
# LOADING CONSTANTS FROM .ENV
# ---------------
# Loading constants and configurations stored in the .env file.
# BEFORE THIS STEP MODIFY THE .env FILE WITH YOUR OWN VALUES

# Load values from .env
load_dotenv()

# Define suffixes
SOLAR_PANEL_SUFFIX = os.getenv('SOLAR_PANEL_SUFFIX', '/PV')  # default value if not found
ELECTRIC_CAR_CHARGER_SUFFIX = os.getenv('ELECTRIC_CAR_CHARGER_SUFFIX', '/ECC')  # default value if not found
BATTERY_SUFFIX = os.getenv('BATTERY_SUFFIX', '/BAT')  # default value if not found

# Define constants
SOURCE_FILE_PATH = os.getenv('SOURCE_FILE_PATH', 'FILE_PATH_HERE')  # default path if not found
EXPORT_FILE_PATH = os.getenv('EXPORT_FILE_PATH', './exports') # default path if not found
PRICE_PER_KWH = float(os.getenv('PRICE_PER_KWH', '183.5'))  # default value if not found
THRESHOLD_FOR_OUTLIERS = float(os.getenv('THRESHOLD_FOR_OUTLIERS', '1.5'))  # default value if not found


### Logger

In [ ]:
logging.getLogger('matplotlib').setLevel(logging.WARNING)

### Task 0: Data Preprocessing

The raw dataset contains date-time information in a non-standard format. In this step:

1. The dataset is loaded into a DataFrame.
2. A new DataFrame is created to hold the processed data.
3. The combined date-time column is split into separate date, start time, and end time values.
4. These separated values are then used to create two new columns: TimePeriodStart and TimePeriodEnd, which represent the start and end of each 15-minute interval in a standard datetime format.
5. The original combined date-time column is dropped.

#### Load the data using the defined function

In [ ]:
processed_data = du.load_and_preprocess_data(SOURCE_FILE_PATH)

#### Extract and process the date and time information

In [ ]:
# Extracting date, start time, and end time from 'Időszeletek'
date = processed_data['Időszeletek'].str.split(' ').str[0]
start_time = processed_data['Időszeletek'].str.split(' - ').str[0].str.split(' ').str[1]
end_time = processed_data['Időszeletek'].str.split(' - ').str[1]

processed_data['TimePeriodStart'] = date + ' ' + start_time
processed_data['TimePeriodEnd'] = date + ' ' + end_time

# Adjust for '24:00' in TimePeriodEnd
mask_24 = processed_data['TimePeriodEnd'].str.endswith('24:00')
processed_data.loc[mask_24, 'TimePeriodEnd'] = (pd.to_datetime(processed_data.loc[mask_24, 'TimePeriodEnd'].str.split(' ').str[0]) + pd.Timedelta(days=1)).dt.strftime('%Y.%m.%d') + ' 00:00'

processed_data['TimePeriodStart'] = pd.to_datetime(processed_data['TimePeriodStart'], format='%Y.%m.%d %H:%M')
processed_data['TimePeriodEnd'] = pd.to_datetime(processed_data['TimePeriodEnd'], format='%Y.%m.%d %H:%M')

processed_data.drop('Időszeletek', axis=1, inplace=True)

# Replace NaN values with 0
processed_data.fillna(0, inplace=True)

#### Add derived columns

In [ ]:
# Extracting DayOfWeek, Month, Hour, and Minute from 'TimePeriodStart'
processed_data['DayOfWeek'] = processed_data['TimePeriodStart'].dt.dayofweek
processed_data['Month'] = processed_data['TimePeriodStart'].dt.month
processed_data['Hour'] = processed_data['TimePeriodStart'].dt.hour
processed_data['Minute'] = processed_data['TimePeriodStart'].dt.minute

#### Identify columns related to different types of meters

In [ ]:
consumption_meter_columns = [col for col in processed_data.columns if SOLAR_PANEL_SUFFIX not in col and ELECTRIC_CAR_CHARGER_SUFFIX not in col and BATTERY_SUFFIX not in col and 'TimePeriod' not in col and col != 'DayOfWeek' and col != 'Month' and col != 'Hour' and col != 'Minute']
solar_panel_columns = [col for col in processed_data.columns if SOLAR_PANEL_SUFFIX in col]
electric_car_charger_columns = [col for col in processed_data.columns if ELECTRIC_CAR_CHARGER_SUFFIX in col]
battery_columns = [col for col in processed_data.columns if BATTERY_SUFFIX in col]

#### Combine consumption meters and solar panels

In [ ]:
# Creating "Pods" dictionary, "Pods" are the pair of Consumption Meters and Solar Panels, not every Consumption Meter has a Solar Panel
pods = {}

for meter in consumption_meter_columns:
    solar_panel = meter + SOLAR_PANEL_SUFFIX
    if solar_panel in solar_panel_columns:
        pods[meter] = (meter, solar_panel)
    else:
        pods[meter] = (meter, None)

#### Variables for analysis

In [ ]:
all_consumption_columns = consumption_meter_columns + electric_car_charger_columns
all_columns_of_interest = all_consumption_columns + solar_panel_columns + battery_columns
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

OLD CODE:

In [ ]:
# # Load data from source file
# processed_data = du.load_and_preprocess_data(SOURCE_FILE_PATH)

# # Extracting date, start time, and end time from 'Időszeletek'
# date = processed_data['Időszeletek'].str.split(' ').str[0]
# start_time = processed_data['Időszeletek'].str.split(' - ').str[0].str.split(' ').str[1]
# end_time = processed_data['Időszeletek'].str.split(' - ').str[1]

# processed_data['TimePeriodStart'] = date + ' ' + start_time
# processed_data['TimePeriodEnd'] = date + ' ' + end_time

# # Adjust for '24:00' in TimePeriodEnd
# mask_24 = processed_data['TimePeriodEnd'].str.endswith('24:00')
# processed_data.loc[mask_24, 'TimePeriodEnd'] = (pd.to_datetime(processed_data.loc[mask_24, 'TimePeriodEnd'].str.split(' ').str[0]) + pd.Timedelta(days=1)).dt.strftime('%Y.%m.%d') + ' 00:00'

# processed_data['TimePeriodStart'] = pd.to_datetime(processed_data['TimePeriodStart'], format='%Y.%m.%d %H:%M')
# processed_data['TimePeriodEnd'] = pd.to_datetime(processed_data['TimePeriodEnd'], format='%Y.%m.%d %H:%M')

# processed_data.drop('Időszeletek', axis=1, inplace=True)

# # Replace NaN values with 0
# processed_data.fillna(0, inplace=True)

# # Extracting DayOfWeek, Month, Hour, and Minute from 'TimePeriodStart'
# processed_data['DayOfWeek'] = processed_data['TimePeriodStart'].dt.dayofweek
# processed_data['Month'] = processed_data['TimePeriodStart'].dt.month
# processed_data['Hour'] = processed_data['TimePeriodStart'].dt.hour
# processed_data['Minute'] = processed_data['TimePeriodStart'].dt.minute

# # Task 3: Distinguish between different meters
# consumption_meter_columns = [col for col in processed_data.columns if SOLAR_PANEL_SUFFIX not in col and ELECTRIC_CAR_CHARGER_SUFFIX not in col and BATTERY_SUFFIX not in col and 'TimePeriod' not in col and col != 'DayOfWeek' and col != 'Month' and col != 'Hour' and col != 'Minute']
# solar_panel_columns = [col for col in processed_data.columns if SOLAR_PANEL_SUFFIX in col]
# electric_car_charger_columns = [col for col in processed_data.columns if ELECTRIC_CAR_CHARGER_SUFFIX in col]
# battery_columns = [col for col in processed_data.columns if BATTERY_SUFFIX in col]

# # Task 4: Create a dictionary for the different meters
# all_consumption_columns = consumption_meter_columns + electric_car_charger_columns
# all_columns_of_interest = all_consumption_columns + solar_panel_columns + battery_columns

# # Task 5: Creating a list of days
# days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# # Creating "Pods" dictionary, "Pods" are the pair of Consumption Meters and Solar Panels, not every Consumption Meter has a Solar Panel
# pods = {}

# for meter in consumption_meter_columns:
#     solar_panel = meter + SOLAR_PANEL_SUFFIX
#     if solar_panel in solar_panel_columns:
#         pods[meter] = (meter, solar_panel)
#     else:
#         pods[meter] = (meter, None)

#### Debug Preprocessing Data

In [ ]:
# Inspect the dataframe
# du.inspect_dataframe(processed_data, 10)

# Testing the pods
print("\nPod Verification:")
for meter, solar_panel in pods.items():
    if solar_panel:
        print(f"Pod: {meter} with Solar Panel: {solar_panel}")
    else:
        print(f"Pod: {meter} without Solar Panel")

#### Downloading the Dataset

In [ ]:
filename = "processed_df"
# du.export_dataframe_to_excel(processed_data, filename, EXPORT_FILE_PATH)

### Task 1: Calculate annual consumption
This task involves understanding the annual consumption of each meter and calculating associated costs:

1. The annual consumption for each meter is computed by summing up its respective values.
2. The annual cost for each meter is derived using a given price per kWh.
3. Average daily consumption is computed for each 15-minute interval throughout the entire year.
4. Average monthly consumption is calculated for each 15-minute interval, grouped by month.
5. A visualization is created to showcase the average consumption per hour throughout the year.

#### 1.1 Calculate the annial consumption and cost for each meter

In [ ]:
# Step 1: Calculate annual consumption for each type of meter
annual_consumption = processed_data[consumption_meter_columns].sum()

# Step 2: Calculate annual costs
annual_costs = annual_consumption * PRICE_PER_KWH

#### 1.2 Claculate the Average Daily Consumption

In [ ]:
# Step 3: Average daily consumption for each 15-minute interval
avg_daily_consumption = processed_data.groupby(['Hour', 'Minute']).mean().reset_index()

#### 1.3 Calculate the Average Monthly Consumption

In [ ]:
# Step 4: Average daily consumption for each 15-minute interval grouped by month
avg_monthly_consumption = processed_data.groupby(['Month', 'Hour', 'Minute']).mean().reset_index()

# Step 5: Additional calculations for plotting
avg_monthly_consumption['Time'] = avg_monthly_consumption['Hour'].astype(str).str.zfill(2) + ':' + avg_monthly_consumption['Minute'].astype(str).str.zfill(2)
avg_monthly_consumption['PlotTime'] = pd.to_datetime('2000-01-01 ' + avg_monthly_consumption['Time'])
avg_monthly_consumption['TotalConsumption'] = avg_monthly_consumption[consumption_meter_columns + electric_car_charger_columns].sum(axis=1)
avg_monthly_consumption['SolarContribution'] = avg_monthly_consumption[solar_panel_columns].sum(axis=1)
avg_monthly_consumption['NetConsumption'] = avg_monthly_consumption['TotalConsumption'] - avg_monthly_consumption['SolarContribution']

#### 1.4 Plot the annual consumption for each meter

In [ ]:
# Break down the plotting into a separate function for clarity
def plot_monthly_consumption(data, consumption_columns, solar_columns, electric_car_columns):
    # Generating time column for plotting
    data['Time'] = data['Hour'].astype(str).str.zfill(2) + ':' + data['Minute'].astype(str).str.zfill(2)
    data['PlotTime'] = pd.to_datetime('2000-01-01 ' + data['Time'])
    
    # Calculate total consumption across all consumption meters and electric car chargers
    data['TotalConsumption'] = data[consumption_columns + electric_car_columns].sum(axis=1)

    # Calculate the difference contributed by the solar panels
    data['SolarContribution'] = data[solar_columns].sum(axis=1)

    # Calculate net consumption (total consumption minus solar panel production)
    data['NetConsumption'] = data['TotalConsumption'] - data['SolarContribution']
    
    # Define the color palette
    palette = sns.color_palette("tab20", len(data['Month'].unique()))

    fig, (ax_total, ax_solar, ax_net) = plt.subplots(3, 1, figsize=(14, 18))

    # Helper function to plot data
    def plot_data(ax, y_value, title):
        for month, color in zip(data['Month'].unique(), palette):
            monthly_data = data[data['Month'] == month]
            ax.plot(monthly_data['PlotTime'], monthly_data[y_value], label=calendar.month_name[month], color=color)
        ax.set_title(title)
        ax.set_xlabel("Hour of Day")
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    # Plotting
    plot_data(ax_total, 'TotalConsumption', "Total Consumption per Hour throughout the Year")
    ax_total.set_ylabel("Average Consumption (kWh)")

    plot_data(ax_solar, 'SolarContribution', "Solar Panel Contribution per Hour throughout the Year")
    ax_solar.set_ylabel("Average Solar Contribution (kWh)")

    plot_data(ax_net, 'NetConsumption', "Net Consumption (after solar contribution) per Hour throughout the Year")
    ax_net.set_ylabel("Average Net Consumption (kWh)")

    plt.tight_layout()
    plt.show()

#### 1.5 Execute the function to plot the average consumption per hour throughout the year

In [ ]:
# Execute the plotting function
plot_monthly_consumption(avg_monthly_consumption, consumption_meter_columns, solar_panel_columns, electric_car_charger_columns)

#### Downloading the Dataset of Task 1

In [ ]:
filename = "task1_annual_consumption"
# du.export_dataframe_to_excel(processed_data, filename, EXPORT_FILE_PATH)

OLD CODE:

In [ ]:
# # Step 1: Calculate annual consumption for each type of meter
# annual_consumption = processed_data[consumption_meter_columns].sum()

# # Step 2: Calculate annual costs
# annual_costs = annual_consumption * PRICE_PER_KWH

# # Step 3: Average daily consumption for each 15-minute interval is already done in preprocessing
# # We have 'Hour' and 'Minute' columns created in the preprocessing step.
# avg_daily_consumption = processed_data.groupby(['Hour', 'Minute']).mean().reset_index()

# # Step 4: Average daily consumption for each 15-minute interval grouped by month
# # 'Month' column was also created during preprocessing.
# avg_monthly_consumption = processed_data.groupby(['Month', 'Hour', 'Minute']).mean().reset_index()

# # Generating time column for plotting
# avg_monthly_consumption['Time'] = avg_monthly_consumption['Hour'].astype(str).str.zfill(2) + ':' + avg_monthly_consumption['Minute'].astype(str).str.zfill(2)
# avg_monthly_consumption['PlotTime'] = pd.to_datetime('2000-01-01 ' + avg_monthly_consumption['Time'])

# # Calculate total consumption across all consumption meters and electric car chargers
# avg_monthly_consumption['TotalConsumption'] = avg_monthly_consumption[consumption_meter_columns + electric_car_charger_columns].sum(axis=1)

# # Calculate the difference contributed by the solar panels
# avg_monthly_consumption['SolarContribution'] = avg_monthly_consumption[solar_panel_columns].sum(axis=1)

# # Calculate net consumption (total consumption minus solar panel production)
# avg_monthly_consumption['NetConsumption'] = avg_monthly_consumption['TotalConsumption'] - avg_monthly_consumption['SolarContribution']

# # Break down the plotting into a separate function for clarity
# def plot_monthly_consumption(data, consumption_columns, solar_columns, electric_car_columns):
#     # Generating time column for plotting
#     data['Time'] = data['Hour'].astype(str).str.zfill(2) + ':' + data['Minute'].astype(str).str.zfill(2)
#     data['PlotTime'] = pd.to_datetime('2000-01-01 ' + data['Time'])
    
#     # Calculate total consumption across all consumption meters and electric car chargers
#     data['TotalConsumption'] = data[consumption_columns + electric_car_columns].sum(axis=1)

#     # Calculate the difference contributed by the solar panels
#     data['SolarContribution'] = data[solar_columns].sum(axis=1)

#     # Calculate net consumption (total consumption minus solar panel production)
#     data['NetConsumption'] = data['TotalConsumption'] - data['SolarContribution']
    
#     # Define the color palette
#     palette = sns.color_palette("tab20", len(data['Month'].unique()))

#     fig, (ax_total, ax_solar, ax_net) = plt.subplots(3, 1, figsize=(14, 18))

#     # Helper function to plot data
#     def plot_data(ax, y_value, title):
#         for month, color in zip(data['Month'].unique(), palette):
#             monthly_data = data[data['Month'] == month]
#             ax.plot(monthly_data['PlotTime'], monthly_data[y_value], label=calendar.month_name[month], color=color)
#         ax.set_title(title)
#         ax.set_xlabel("Hour of Day")
#         ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
#         ax.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
#         ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

#     # Plotting
#     plot_data(ax_total, 'TotalConsumption', "Total Consumption per Hour throughout the Year")
#     ax_total.set_ylabel("Average Consumption (kWh)")

#     plot_data(ax_solar, 'SolarContribution', "Solar Panel Contribution per Hour throughout the Year")
#     ax_solar.set_ylabel("Average Solar Contribution (kWh)")

#     plot_data(ax_net, 'NetConsumption', "Net Consumption (after solar contribution) per Hour throughout the Year")
#     ax_net.set_ylabel("Average Net Consumption (kWh)")

#     plt.tight_layout()
#     plt.show()

# # Execute the plotting function
# plot_monthly_consumption(avg_monthly_consumption, consumption_meter_columns, solar_panel_columns, electric_car_charger_columns)

### Task 2: Daily Consumption Patterns and Monthly Breakdowns

This task aims to identify patterns in consumption across different days of the week:

1. Average consumption values for each day of the week are computed.
2. A bar chart is created to showcase the average consumption for each day, from Monday to Sunday.

#### 2.1 Calculations for 15-min, hourly and daily average consumption

In [ ]:
# 1. Visual: Calculate average 15-minute consumption for the days of the week for each month
avg_15min_weekly_monthly_consumption = processed_data.groupby(['Month', 'DayOfWeek']).mean()[all_consumption_columns].reset_index()
avg_15min_weekly_monthly_consumption['DayOfWeekName'] = avg_15min_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# 2. Visual: Calculate average hourly consumption for the days of the week for each month
hourly_data_without_datetime = processed_data.drop(columns=['TimePeriodStart', 'TimePeriodEnd'], errors='ignore')
hourly_sum = hourly_data_without_datetime.groupby(['Month', 'DayOfWeek', 'Hour']).sum()[all_consumption_columns].reset_index()
avg_hourly_weekly_monthly_consumption = hourly_sum.groupby(['Month', 'DayOfWeek']).mean().reset_index()
avg_hourly_weekly_monthly_consumption['DayOfWeekName'] = avg_hourly_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# 3. Visual: Calculate average daily consumption for the days of the week for each month
data_without_datetime = processed_data.drop(columns=['TimePeriodStart', 'TimePeriodEnd'], errors='ignore')
daily_sum = data_without_datetime.groupby(['Month', 'DayOfWeek']).sum()[all_consumption_columns].reset_index()
avg_daily_weekly_monthly_consumption = daily_sum.groupby(['Month', 'DayOfWeek']).mean().reset_index()
avg_daily_weekly_monthly_consumption['DayOfWeekName'] = avg_daily_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])


#### 2.1.1   Visual for 15-min interval average consumption

In [ ]:
def plot_monthly_consumption_adjusted(data, consumption_columns):
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15), sharex=True, sharey=True)
    days_order = list(calendar.day_name)

    for month in range(1, 13):
        ax = axes[(month-1) // 4][(month-1) % 4]
        
        # Aggregate by day of the week within each month
        monthly_data = data[data['Month'] == month].groupby(['DayOfWeek', 'DayOfWeekName']).mean().reset_index().sort_values(by="DayOfWeek")
        
        # Plot stacked bar chart
        monthly_data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
        # Set bar labels
        for patch in ax.patches:
            width, height = patch.get_width(), patch.get_height()
            x, y = patch.get_xy()
            ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
                        ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
                        textcoords='offset points')

        ax.set_title(calendar.month_name[month])
        ax.set_xticks(list(range(7)))  # Set x-ticks for every day
        ax.set_xticklabels(days_order, rotation=45, ha='right')  
        ax.grid(axis='y')
        ax.set_ylabel('Avg 15-min interval Consumption (kWh)')

    # Adjust the legend to avoid overlapping
    fig.legend(consumption_columns, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=len(consumption_columns))

    plt.tight_layout()
    plt.show()

#### 2.1.2   Visual for hourly and daily average consumption

In [ ]:
def plot_stacked_consumption(data, consumption_columns, title_prefix, chart_type="hourly"):
    fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15), sharex=True, sharey=True)
    days_order = list(calendar.day_name)
    
    if chart_type == "daily":
        ax = axes[0][0]
        # Create the stacked bar chart
        data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
        # Set bar labels
        for patch in ax.patches:
            width, height = patch.get_width(), patch.get_height()
            x, y = patch.get_xy()
            ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
                        ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
                        textcoords='offset points')

        ax.set_title(f"Average {title_prefix} Consumption")
        ax.set_xticks(list(range(7)))  # Set x-ticks for every day
        ax.set_xticklabels(days_order, rotation=45, ha='right')  
        ax.grid(axis='y')
        ax.set_ylabel(f'Avg {title_prefix} Consumption (kWh)')
        # Hide other subplots
        for i in range(3):
            for j in range(4):
                if i != 0 or j != 0:
                    axes[i][j].axis('off')
        plt.tight_layout()
        plt.show()
        return

    for month in range(1, 13):
        ax = axes[(month-1) // 4][(month-1) % 4]
        monthly_data = data[data['Month'] == month].sort_values(by="DayOfWeek")
        
        # If there's no data for the month, continue
        if monthly_data.empty:
            continue
        
        # Create the stacked bar chart
        monthly_data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
        # Set bar labels
        for patch in ax.patches:
            width, height = patch.get_width(), patch.get_height()
            x, y = patch.get_xy()
            ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
                        ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
                        textcoords='offset points')

        ax.set_title(calendar.month_name[month])
        ax.set_xticks(list(range(7)))  # Set x-ticks for every day
        ax.set_xticklabels(days_order, rotation=45, ha='right')  
        ax.grid(axis='y')
        ax.set_ylabel(f'Avg {title_prefix} Consumption (kWh)')

    # Adjust the legend to avoid overlapping
    fig.legend(consumption_columns, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=len(consumption_columns))

    plt.tight_layout()
    plt.show()

#### 2.1.3   Execute functions for plotting the average consumption for each day of the week

In [ ]:
# Plotting the data
# plot_monthly_consumption_adjusted(avg_15min_weekly_monthly_consumption, all_consumption_columns)
plot_monthly_consumption_adjusted(avg_15min_weekly_monthly_consumption, all_consumption_columns)
plot_stacked_consumption(avg_hourly_weekly_monthly_consumption, all_consumption_columns, "Hourly", chart_type="monthly")
plot_stacked_consumption(avg_daily_weekly_monthly_consumption, all_consumption_columns, "Daily", chart_type="monthly")

#### Downloading the Dataset of Task 2

In [ ]:
# Exporting the 15-minute interval averages
filename_15min = "task2_15min_weekly_monthly_consumption"
# du.export_dataframe_to_excel(avg_15min_weekly_monthly_consumption, filename_15min, EXPORT_FILE_PATH)

# Exporting the hourly averages
filename_hourly = "task2_hourly_weekly_monthly_consumption"
# du.export_dataframe_to_excel(avg_hourly_weekly_monthly_consumption, filename_hourly, EXPORT_FILE_PATH)

# Exporting the daily averages
filename_daily = "task2_daily_weekly_monthly_consumption"
# du.export_dataframe_to_excel(avg_daily_weekly_monthly_consumption, filename_daily, EXPORT_FILE_PATH)

#### 2.2 Analyzing Daily Average Consumption

This section dives deeper into daily consumption patterns. Specifically, it aims to provide insights into how consumption varies across different days of the week by examining the daily average consumption for each column of interest.

***Steps:***
1. ***Hourly Summation:***
   - For each day of the week and each hour of the day, the data is grouped and summed. This step provides the total consumption for each hour.

2. ***Calculate Daily Averages from Hourly Data:***
   - After obtaining the hourly sums, the next step is to sum these values to get a daily total.
This daily total is then divided by 24 (the number of hours in a day) to compute the daily average consumption for each column of interest.

3. ***Visualization:***
   - A bar chart is plotted to showcase the daily average consumption for each day of the week.
Each column of interest is represented as a separate bar, making it easier to compare the consumption values for different columns on any given day.
The x-axis represents the days of the week, and the y-axis shows the average consumption in kWh.
A legend is provided to distinguish between different columns.
By examining the chart, one can glean insights into how consumption habits might change depending on the day of the week.

#### 2.2.1 Calculate the daily average consumption for each meter

In [ ]:
# Group by the day of the week and hour, then calculate the sum for specific columns
hourly_sum = processed_data.groupby(['DayOfWeek', 'Hour'])[all_columns_of_interest].sum().reset_index()

# Calculate daily averages from hourly summed data
daily_sum = hourly_sum.groupby('DayOfWeek')[all_columns_of_interest].sum().reset_index()
daily_avg_from_sum = daily_sum.copy()

# Divide the summed values by 24 to get the daily average
for col in all_columns_of_interest:
    daily_avg_from_sum[col] = daily_sum[col] / 24

#### 2.2.2 Visualize the daily average consumption for each meter

In [ ]:
plt.figure(figsize=(18, 10))

# Calculate the number of bars and their width
num_bars = len(all_columns_of_interest)
bar_width = 0.15
positions = np.arange(len(days))

for i, col in enumerate(all_columns_of_interest):
    plt.bar(positions + i*bar_width, daily_avg_from_sum[col], width=bar_width, label=col)

# Setting the x-axis
plt.xticks(positions + bar_width * (num_bars - 1) / 2, days)
plt.xlabel('Day of the Week')
plt.ylabel('Average Value (kWh)')
plt.title("Daily Average Consumption Values for Each Column")
plt.legend(title='Columns', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', which="both", ls="--", c='0.7')
plt.tight_layout()
plt.show()

#### 2.3: Hourly Consumption Patterns
In this section, the goal is to analyze the hourly consumption patterns for each day of the week. The data is broken down into hourly segments to identify any trends or patterns in consumption during different times of the day.

***Steps:***

1. ***Filter Relevant Columns:***
    - From the entire dataset, only those columns that are of interest and present in the processed data are selected.


2. ***Hourly Average Computation:***
    - The data is grouped by both the day of the week and the hour. For each grouping, the average consumption for the columns of interest is calculated.

3. ***Day Labeling:***
    - The numeric representation of the days (0-6) is converted into string labels (Monday to Sunday) using the 'days' list, making the data more interpretable.
4. ***Visualization:***
    - A function named plot_column_data is defined to plot the hourly average consumption for each column.
    - The function utilizes Seaborn's barplot to visually represent the hourly consumption trends for each day of the week.
    - The x-axis represents the hours of the day, while the y-axis shows the average consumption in kWh.
    - A separate chart is plotted for each column, making it easier to analyze individual consumption trends.


By reviewing these charts, one can discern patterns in consumption for each hour of the day, and understand if there are any particular hours where consumption peaks or drops.

In [ ]:
# Group by the day of the week and hour, then calculate the mean for the columns of interest
numeric_cols = [col for col in all_columns_of_interest if col in processed_data.columns]
hourly_avg = processed_data.groupby(['DayOfWeek', 'Hour'])[numeric_cols].mean().reset_index()

# Convert DayOfWeek to string labels for clarity in plotting using the 'days' list
hourly_avg['DayOfWeek'] = hourly_avg['DayOfWeek'].apply(lambda x: days[x])

In [ ]:
# Helper function to plot data for each column
def plot_column_data(column_name):
    plt.figure(figsize=(14, 6))
    
    # Use Seaborn's barplot
    sns.barplot(data=hourly_avg, x=hourly_avg['Hour'].astype(str), y=column_name, hue='DayOfWeek', errorbar=None, palette='viridis')
    
    plt.title(f"Hourly Average Consumption for {column_name}")
    plt.legend(title='Day of the Week', labels=days, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, which="both", ls="--", c='0.7')
    plt.xticks(list(range(0, 24)), list(range(0, 24)))  # Set the x-ticks explicitly
    plt.ylabel(f'Average {column_name}')
    plt.xlabel('Hour of Day')
    plt.tight_layout()
    plt.show()

In [ ]:
# Plotting data for each column of interest
for column in numeric_cols:
    plot_column_data(column)

OLD CODE:

In [ ]:
# # Calculate average consumption for the days of the week for each month
# avg_weekly_monthly_consumption = processed_data.groupby(['Month', 'DayOfWeek']).mean()[all_consumption_columns].reset_index()

# # Convert numerical day representation to actual day names
# avg_weekly_monthly_consumption['DayOfWeekName'] = avg_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# def plot_monthly_consumption_adjusted(data, consumption_columns):
#     fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15), sharex=True, sharey=True)
#     days_order = list(calendar.day_name)

#     for month in range(1, 13):
#         ax = axes[(month-1) // 4][(month-1) % 4]
#         monthly_data = data[data['Month'] == month].sort_values(by="DayOfWeek")
        
#         bars = []
#         for column in consumption_columns:
#             bars.append(ax.bar(pd.Categorical(monthly_data['DayOfWeekName'], categories=days_order), monthly_data[column], label=column, alpha=0.7))
        
#         # Set bar labels
#         for bar_set in bars:
#             for bar in bar_set:
#                 yval = bar.get_height()
#                 ax.text(bar.get_x() + bar.get_width()/2, yval + 0.1, round(yval,2), ha='center', va='bottom', fontsize=9)
        
#         ax.set_title(calendar.month_name[month])
#         ax.set_xticks(list(range(7)))  # Set x-ticks for every day
#         ax.set_xticklabels(days_order, rotation=45, ha='right')  
#         ax.grid(axis='y')
#         ax.set_ylabel('Avg Consumption (kWh)')

#     # Adjust the legend to avoid overlapping
#     fig.legend(consumption_columns, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=len(consumption_columns))

#     plt.tight_layout()
#     plt.show()
# plot_monthly_consumption_adjusted(avg_weekly_monthly_consumption, all_consumption_columns)

In [ ]:
# # 1. Visual: Calculate average 15-minute consumption for the days of the week for each month
# avg_15min_weekly_monthly_consumption = processed_data.groupby(['Month', 'DayOfWeek']).mean()[all_consumption_columns].reset_index()
# avg_15min_weekly_monthly_consumption['DayOfWeekName'] = avg_15min_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# # 2. Visual: Calculate average hourly consumption for the days of the week for each month
# avg_hourly_weekly_consumption = processed_data.groupby(['Month', 'DayOfWeek', 'Hour']).mean()[all_consumption_columns].reset_index()
# avg_hourly_weekly_monthly_consumption = avg_hourly_weekly_consumption.groupby(['Month', 'DayOfWeek']).mean().reset_index()
# avg_hourly_weekly_monthly_consumption['DayOfWeekName'] = avg_hourly_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# # 3. Visual: Calculate average daily consumption for the days of the week for each month
# # Removing datetime columns
# data_without_datetime = processed_data.drop(columns=['TimePeriodStart', 'TimePeriodEnd'], errors='ignore')

# # Summarizing values by day
# daily_sum = data_without_datetime.groupby(['Month', 'DayOfWeek']).sum()[all_consumption_columns].reset_index()

# # Calculating the average of the same days in the same month
# avg_daily_weekly_monthly_consumption = daily_sum.groupby(['Month', 'DayOfWeek']).mean().reset_index()
# avg_daily_weekly_monthly_consumption['DayOfWeekName'] = avg_daily_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])


# def plot_stacked_consumption(data, consumption_columns, title_prefix, chart_type="hourly"):
#     fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15), sharex=True, sharey=True)
#     days_order = list(calendar.day_name)
    
#     if chart_type == "daily":
#         ax = axes[0][0]
#         # Create the stacked bar chart
#         data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
#         # Set bar labels
#         for patch in ax.patches:
#             width, height = patch.get_width(), patch.get_height()
#             x, y = patch.get_xy()
#             ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
#                         ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
#                         textcoords='offset points')

#         ax.set_title(f"Average {title_prefix} Consumption")
#         ax.set_xticks(list(range(7)))  # Set x-ticks for every day
#         ax.set_xticklabels(days_order, rotation=45, ha='right')  
#         ax.grid(axis='y')
#         ax.set_ylabel(f'Avg {title_prefix} Consumption (kWh)')
#         # Hide other subplots
#         for i in range(3):
#             for j in range(4):
#                 if i != 0 or j != 0:
#                     axes[i][j].axis('off')
#         plt.tight_layout()
#         plt.show()
#         return

#     for month in range(1, 13):
#         ax = axes[(month-1) // 4][(month-1) % 4]
#         monthly_data = data[data['Month'] == month].sort_values(by="DayOfWeek")
        
#         # If there's no data for the month, continue
#         if monthly_data.empty:
#             continue
        
#         # Create the stacked bar chart
#         monthly_data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
#         # Set bar labels
#         for patch in ax.patches:
#             width, height = patch.get_width(), patch.get_height()
#             x, y = patch.get_xy()
#             ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
#                         ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
#                         textcoords='offset points')

#         ax.set_title(calendar.month_name[month])
#         ax.set_xticks(list(range(7)))  # Set x-ticks for every day
#         ax.set_xticklabels(days_order, rotation=45, ha='right')  
#         ax.grid(axis='y')
#         ax.set_ylabel(f'Avg {title_prefix} Consumption (kWh)')

#     # Adjust the legend to avoid overlapping
#     fig.legend(consumption_columns, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=len(consumption_columns))

#     plt.tight_layout()
#     plt.show()

# # Plotting the data
# plot_stacked_consumption(avg_weekly_monthly_consumption, all_consumption_columns, "15-min Interval", chart_type="monthly")
# plot_stacked_consumption(avg_hourly_weekly_monthly_consumption, all_consumption_columns, "Hourly", chart_type="monthly")


In [ ]:
# # Removing datetime columns
# data_without_datetime = processed_data.drop(columns=['TimePeriodStart', 'TimePeriodEnd'], errors='ignore')

# # Summarizing values by day
# daily_sum = data_without_datetime.groupby(['Month', 'DayOfWeek']).sum()[all_consumption_columns].reset_index()

# # Calculating the average of the same days in the same month
# avg_daily_weekly_monthly_consumption = daily_sum.groupby(['Month', 'DayOfWeek']).mean().reset_index()
# avg_daily_weekly_monthly_consumption['DayOfWeekName'] = avg_daily_weekly_monthly_consumption['DayOfWeek'].apply(lambda x: calendar.day_name[x])

# def plot_monthly_daily_consumption(data, consumption_columns):
#     fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15), sharex=True, sharey=True)
#     days_order = list(calendar.day_name)
    
#     for month in range(1, 13):
#         ax = axes[(month-1) // 4][(month-1) % 4]
#         monthly_data = data[data['Month'] == month].sort_values(by="DayOfWeek")
        
#         # If there's no data for the month, continue
#         if monthly_data.empty:
#             continue
        
#         # Create the stacked bar chart
#         monthly_data.set_index('DayOfWeekName')[consumption_columns].plot(kind='bar', stacked=True, ax=ax, alpha=0.7)
        
#         # Set bar labels
#         for patch in ax.patches:
#             width, height = patch.get_width(), patch.get_height()
#             x, y = patch.get_xy()
#             ax.annotate(f'{height:.2f}', (x + width / 2., y + height / 2.),
#                         ha='center', va='center', fontsize=9, color='black', xytext=(0, -10),
#                         textcoords='offset points')

#         ax.set_title(calendar.month_name[month])
#         ax.set_xticks(list(range(7)))  # Set x-ticks for every day
#         ax.set_xticklabels(days_order, rotation=45, ha='right')  
#         ax.grid(axis='y')
#         ax.set_ylabel(f'Avg Consumption (kWh)')

#     # Adjust the legend to avoid overlapping
#     fig.legend(consumption_columns, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=len(consumption_columns))

#     plt.tight_layout()
#     plt.show()

# # Call the function
# plot_monthly_daily_consumption(avg_daily_weekly_monthly_consumption, all_consumption_columns)


In [ ]:
# # Group by the day of the week and hour, then calculate the sum for specific columns
# hourly_sum = processed_data.groupby(['DayOfWeek', 'Hour'])[all_columns_of_interest].sum().reset_index()

# # Calculate daily averages from hourly summed data
# daily_sum = hourly_sum.groupby('DayOfWeek')[all_columns_of_interest].sum().reset_index()
# daily_avg_from_sum = daily_sum.copy()

# # Divide the summed values by 24 to get the daily average
# for col in all_columns_of_interest:
#     daily_avg_from_sum[col] = daily_sum[col] / 24

# # Plotting
# plt.figure(figsize=(18, 10))

# # Calculate the number of bars and their width
# num_bars = len(all_columns_of_interest)
# bar_width = 0.15
# positions = np.arange(len(days))

# for i, col in enumerate(all_columns_of_interest):
#     plt.bar(positions + i*bar_width, daily_avg_from_sum[col], width=bar_width, label=col)

# # Setting the x-axis
# plt.xticks(positions + bar_width * (num_bars - 1) / 2, days)
# plt.xlabel('Day of the Week')
# plt.ylabel('Average Value (kWh)')
# plt.title("Daily Average Consumption Values for Each Column")
# plt.legend(title='Columns', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.grid(axis='y', which="both", ls="--", c='0.7')
# plt.tight_layout()
# plt.show()

In [ ]:
print(processed_data.columns)
print(processed_data.dtypes)

In [ ]:
# # Group by the day of the week and hour, then calculate the mean for the columns of interest
# numeric_cols = [col for col in all_columns_of_interest if col in processed_data.columns]
# hourly_avg = processed_data.groupby(['DayOfWeek', 'Hour'])[numeric_cols].mean().reset_index()

# # Convert DayOfWeek to string labels for clarity in plotting
# day_mapping = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
# hourly_avg['DayOfWeek'] = hourly_avg['DayOfWeek'].replace(day_mapping)

# # Ensure 'Hour' column is of integer type
# hourly_avg['Hour'] = hourly_avg['Hour'].astype(int)

# # Helper function to plot data for each column
# def plot_column_data(column_name):
#     plt.figure(figsize=(14, 6))
    
#     # Use Seaborn's barplot
#     sns.barplot(data=hourly_avg, x='Hour', y=column_name, hue='DayOfWeek', errorbar=None, palette='viridis')
    
#     plt.title(f"Hourly Average Consumption for {column_name}")
#     plt.legend(title='Day of the Week', labels=list(day_mapping.values()), bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.grid(True, which="both", ls="--", c='0.7')
#     plt.xticks(list(range(0, 24)))
#     plt.ylabel(f'Average {column_name}')
#     plt.xlabel('Hour of Day')
#     plt.tight_layout()
#     plt.show()

# # Plotting data for each column of interest
# for column in numeric_cols:
#     plot_column_data(column)


### Task 3: Examining Intra-day Consumption Distribution
The goal of this task is to observe consumption patterns within a day. By studying consumption trends at 15-minute intervals, we can identify periods of peak consumption and more.

***Steps:***

1. ***Calculate average consumption for each 15-minute interval of the day***
    - We aim to compute the average consumption for each quarter-hour of the day. To achieve this, we'll leverage the 'Hour' and 'Minute' columns we extracted during preprocessing.


2. ***Visualization of intra-day consumption patterns***
   - In this step, we will visualize the computed average consumption values for each 15-minute interval throughout the day. This will provide insights into consumption trends within a day, helping identify peak and off-peak times.

In [ ]:
# Step 1: Calculate average consumption for each 15-minute interval of the day

# As we have already extracted 'Hour' and 'Minute' in the preprocessing step, we can directly compute the average.
avg_intra_day = processed_data.groupby(['Hour', 'Minute']).mean()[all_consumption_columns].reset_index()

# Convert 'Hour' and 'Minute' to a time format for better visualization
avg_intra_day['Time'] = avg_intra_day['Hour'].astype(str).str.zfill(2) + ':' + avg_intra_day['Minute'].astype(str).str.zfill(2)

# Step 2: Visualization of intra-day consumption patterns

plt.figure(figsize=(16, 8))

# Plot data for each consumption column
for column in all_consumption_columns:
    plt.plot(avg_intra_day['Time'], avg_intra_day[column], label=column, marker='o', markersize=3)

# Set labels, title, and format the x-axis
plt.xlabel('Time of Day')
plt.ylabel('Average Consumption (kWh)')
plt.title('Intra-day Distribution of Average Consumption (2022)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y')

plt.tight_layout()
plt.show()

### Task 4: Identifying Days with Peak Consumption at 15-minute Intervals
The objective of this task is to spot days with peak power usage. We'll be focusing on 15-minute intervals to determine which specific periods have the highest consumption. This information is crucial for optimizing power consumption and planning for peak demand periods.

***Steps:***

1. ***Convert power readings to energy consumption***
    - Given the data is in kW (power) for 15-minute intervals, we'll multiply by 4 to convert these readings to kWh (energy).
Calculate the total consumption for each 15-minute interval

2. ***Summing up the consumption values across all columns for each 15-minute interval helps identify peak times.***
   - Identify and highlight peak consumption periods
   - Using the computed total consumption for each interval, we'll determine which time frame had the highest consumption.
   - Visualization of intra-day consumption patterns with peak highlighted
   - A line plot will provide a comprehensive view of the consumption trends throughout the day. The peak consumption period will be highlighted to offer an immediate view of the time with the highest demand.

#### 4.1 Calculations for the peak consumption periods

In [ ]:
# Step 1: Convert to kWh
# Multiplying by 4 to convert kW readings of 15-minute intervals to kWh
avg_intra_day[all_consumption_columns] *= 4

# Step 2: Calculate total consumption for each time interval
avg_intra_day['Total'] = avg_intra_day[all_consumption_columns].sum(axis=1)

# Step 3: Identify peak consumption time
peak_time = avg_intra_day.iloc[avg_intra_day['Total'].idxmax()]['Time']
peak_value = avg_intra_day['Total'].max()

#### 4.2 Visual for the peak consumption periods

In [ ]:
# Visualization
plt.figure(figsize=(16, 8))

# Plot data for each consumption column
for column in all_consumption_columns:
    plt.plot(avg_intra_day['Time'].astype(str), avg_intra_day[column], label=column, marker='o', markersize=3)

# Highlight the peak consumption period
plt.axvline(peak_time, color='red', linestyle='--', label=f'Peak Consumption at {peak_time}')
plt.axhline(peak_value, color='green', linestyle='--', label=f'Peak Consumption: {peak_value:.2f} kWh')

# Set labels, title, and format the x-axis
plt.xlabel('Time of Day')
plt.ylabel('Average Consumption (kWh)')
plt.title('Intra-day Distribution of Average Consumption with Peak Highlighted (2022)')
plt.xticks(avg_intra_day['Time'].astype(str).unique(), rotation=45, ha='right')
plt.legend()
plt.grid(axis='y')

plt.tight_layout()
plt.show()

### Task 5: Identify the top 5 days with the highest instantaneous power.

The focus of this task is to pinpoint periods of peak consumption:

1. **Highlighting Peak Intervals**:
    - The dataset is first scoured to identify the 15-minute intervals with the top 5 highest total power consumptions. It's worth noting that multiple peak values might occur on the same day. The visual representation of this data will underscore these peak values, allowing us to identify the most power-intensive moments throughout the year.
2. **Analysis of Unique Peak Days**:
    - To get a broader view, we delve deeper to identify the top 5 unique days that experienced the highest instantaneous power. This approach ensures we're not just looking at isolated peak intervals but rather entire days of significant power usage.
    - A line plot is then crafted to vividly display the intra-day power distribution for each of these standout days, painting a clear picture of consumption dynamics on these particularly demanding days.

By the culmination of this task, we'll have a nuanced understanding of both instantaneous power spikes and broader days of high consumption, offering a multifaceted view of peak usage periods.

#### 5.1 The top 5 days with the highest instantaneous power

##### 5.1.1 Calculations for the top 5 days with the highest instantaneous power

In [ ]:
# Step 1: Convert consumption to power (kW)
# Since the data is for 15-minute intervals, we multiply by 4 to get kW
processed_data[all_consumption_columns] *= 4

# Calculate total power for each interval
processed_data['TotalPower'] = processed_data[all_consumption_columns].sum(axis=1)

# Step 2: Identify top 5 intervals with the highest power
top_5_intervals = processed_data.nlargest(5, 'TotalPower')

# Step 3: Extract unique dates from top intervals
unique_top_dates = top_5_intervals['TimePeriodStart'].dt.date.unique()

print("Top 5 Dates with the Highest Instantaneous Power:")
for date in unique_top_dates:
    print(date)

##### 5.1.2 Visual for the top 5 days with the highest instantaneous power

In [ ]:
# Step 4: Visualization

plt.figure(figsize=(16, 10))

# Plotting intra-day distribution for each unique top date
for date in unique_top_dates:
    subset = processed_data[processed_data['TimePeriodStart'].dt.date == date]
    time = subset['TimePeriodStart'].dt.strftime('%H:%M')
    plt.plot(time, subset['TotalPower'], label=str(date), marker='o', markersize=4)

# Adjusting the plot
plt.xlabel('Time of Day')
plt.ylabel('Power (kW)')
plt.title('Intra-day Power Distribution for Days with Highest Instantaneous Power')
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis='y')
plt.tight_layout()

plt.show()

#### 5.2 The top 5 UNIQUE days with the highest instantaneous power

##### 5.2.1 Calculations for the top 5 UNIQUE days with the highest instantaneous power

In [ ]:
# Calculate daily total power
daily_total_power = processed_data.groupby(processed_data['TimePeriodStart'].dt.date)['TotalPower'].sum()

# Get top 5 unique dates with the highest total power
top_5_dates = daily_total_power.nlargest(5).index

print("Top 5 Dates with the Highest Instantaneous Power:")
for date in top_5_dates:
    print(date)

##### 5.2.2 Visual for the top 5 UNIQUE days with the highest instantaneous power

In [ ]:
# Visualization
plt.figure(figsize=(16, 10))

# Plotting intra-day distribution for each unique top date
for date in top_5_dates:
    subset = processed_data[processed_data['TimePeriodStart'].dt.date == date]
    time = subset['TimePeriodStart'].dt.strftime('%H:%M')
    plt.plot(time, subset['TotalPower'], label=str(date), marker='o', markersize=4)

# Adjusting the plot
plt.xlabel('Time of Day')
plt.ylabel('Power (kW)')
plt.title('Intra-day Power Distribution for Days with Highest Instantaneous Power')
plt.xticks(rotation=45)
plt.legend()
plt.grid(axis='y')
plt.tight_layout()

plt.show()

#### 5.3 Detailed Intra-day Power Distribution for Peak Consumption Days
This task focuses on understanding the power distribution for the top 5 days with the highest consumption. By studying these days closely, we can identify any unique patterns or anomalies that might occur during peak times.

In [ ]:
# Visualization
fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(16, 30))
all_consumption_columns = consumption_meter_columns + electric_car_charger_columns

for idx, date in enumerate(top_5_dates):
    ax = axes[idx]
    
    # Subset for the date
    subset = processed_data[processed_data['TimePeriodStart'].dt.date == date]
    
    # Extract data for each consumption column and stack them
    bar_data = [subset[col].values for col in all_consumption_columns]
    cum_data = np.cumsum(bar_data, axis=0)
    
    for i, col in enumerate(all_consumption_columns):
        if i == 0:
            ax.bar(subset['TimePeriodStart'].dt.strftime('%H:%M'), bar_data[i], label=col)
        else:
            ax.bar(subset['TimePeriodStart'].dt.strftime('%H:%M'), bar_data[i], bottom=cum_data[i - 1], label=col)
    
    # Adjusting the subplot
    ax.set_title(f'Intra-day Power Distribution for {date}')
    ax.set_xlabel('Time of Day')
    ax.set_ylabel('Power (kW)')
    time_labels = subset['TimePeriodStart'].dt.strftime('%H:%M')
    ax.set_xticks(np.arange(len(time_labels)))
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(axis='y')

plt.tight_layout()
plt.show()

#### 5.4 Detailed Examination of Power Distribution on Specific Dates
The objective of this task is to conduct a granular analysis of the power distribution at 15-minute intervals on specified dates. By focusing on particular days, we can gain deeper insights into consumption patterns and deviations from typical behaviors.

##### 5.4.1 Dates of interests

In [ ]:
# Dates of interest
dates_of_interest = ['2022-01-24', '2022-07-04']

##### 5.4.2 Visual for the power distribution on specific dates

In [ ]:
# Visualization
fig, axes = plt.subplots(nrows=len(dates_of_interest), ncols=1, figsize=(16, 15))  # Adjusted the figure size

for idx, date in enumerate(dates_of_interest):
    ax = axes[idx]
    
    # Subset for the date
    subset = processed_data[processed_data['TimePeriodStart'].dt.date == pd.to_datetime(date).date()]
    
    # Extract data for each consumption column and stack them
    bar_data = [subset[col].values for col in all_consumption_columns]
    cum_data = np.cumsum(bar_data, axis=0)
    
    for i, col in enumerate(all_consumption_columns):
        if i == 0:
            ax.bar(subset['TimePeriodStart'].dt.strftime('%H:%M'), bar_data[i], label=col)
        else:
            ax.bar(subset['TimePeriodStart'].dt.strftime('%H:%M'), bar_data[i], bottom=cum_data[i - 1], label=col)
    
    # Adjusting the subplot
    ax.set_title(f'Intra-day Power Distribution for {date}')
    ax.set_xlabel('Time of Day')
    ax.set_ylabel('Power (kW)')
    time_labels = subset['TimePeriodStart'].dt.strftime('%H:%M')
    ax.set_xticks(np.arange(len(time_labels)))
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(axis='y')

plt.tight_layout()
plt.show()

### Task 6: Visualizing Quarterly Consumption

This task aims to understand the distribution of daily energy consumption for each meter:

1. Histograms are created for each meter to show the distribution of daily energy consumption values.
2. These histograms provide insights into the frequency of different daily consumption values for each meter, highlighting common consumption patterns and any anomalies.

#### 6.1 Calculations for the quarterly consumption

In [ ]:
# Compute the daily energy consumption for each meter
processed_data['Date'] = processed_data['TimePeriodStart'].dt.date
daily_consumption = processed_data.groupby('Date')[all_consumption_columns].sum()

#### 6.2 Visual for the quarterly consumption

In [ ]:
# Setting up the plotting environment
sns.set_style("whitegrid")
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))

# Plotting histograms for each meter
for idx, column in enumerate(all_consumption_columns):
    sns.histplot(daily_consumption[column], ax=axes[idx], bins=30, kde=True, color='skyblue', edgecolor='black')
    
    # Setting titles and labels for each subplot
    axes[idx].set_title(f'Distribution of Daily Energy Consumption for {column}')
    axes[idx].set_xlabel('Daily Energy Consumption (kWh)')
    axes[idx].set_ylabel('Frequency')

# Adjusting the layout to prevent overlap
plt.tight_layout()

# Displaying the plots
plt.show()

### Task 7: Anomaly Detection in Daily Energy Consumption

Detecting anomalies in energy consumption is vital for various reasons, such as identifying unusual energy usage patterns, equipment malfunctions, or potential fraud. In this task, we'll employ the Interquartile Range (IQR) method to pinpoint anomalies in our daily energy consumption dataset.

***Steps:***

1. **Threshold Calculation**:
    - For each meter (or consumption column), the first quartile (Q1) and the third quartile (Q3) of daily consumption are computed.
    - The interquartile range (IQR) is then determined as \( IQR = Q3 - Q1 \).
    - To identify potential outliers, thresholds are set as 1.5 times the IQR below Q1 and above Q3.
    - Data points outside of these bounds are considered anomalies.

2. **Identify Anomalous Dates**:
    - The previously calculated thresholds are used to determine the specific dates where the consumption was anomalously low or high.
    - For each meter, the dates outside the threshold bounds are extracted and stored.

3. **Visualization of Anomalies**:
    - Daily energy consumption is plotted over time.
    - Detected anomalies are highlighted in red to differentiate them from the regular consumption patterns.
    - This visual representation helps in intuitively understanding the days with unusual consumption patterns.


##### 7.1 Calculating Thresholds for Anomaly Detection**

To detect anomalies, we first need to define what constitutes an "anomaly." In this cell, we:
- Calculate the first (Q1) and third quartiles (Q3) for the consumption data of each meter.
- Determine the Interquartile Range (IQR) as \( IQR = Q3 - Q1 \).
- Set thresholds at 1.5 times the IQR below Q1 and above Q3. Any value outside these bounds can be considered an anomaly.
- Display the calculated thresholds for each meter.

In [ ]:
# For each meter, calculate Q1, Q3, IQR, and the bounds for outliers
thresholds = {}

for column in all_consumption_columns:
    Q1 = daily_consumption[column].quantile(0.25)
    Q3 = daily_consumption[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - THRESHOLD_FOR_OUTLIERS * IQR
    upper_bound = Q3 + THRESHOLD_FOR_OUTLIERS * IQR
    
    thresholds[column] = {"Lower Bound": lower_bound, "Upper Bound": upper_bound}

# Display thresholds for each meter
for meter, bounds in thresholds.items():
    print(f"Thresholds for {meter}:")
    print(f"  - Lower Bound: {bounds['Lower Bound']:.2f}")
    print(f"  - Upper Bound: {bounds['Upper Bound']:.2f}")
    print("\n")


##### 7.2 Identifying Dates with Anomalies

Using the thresholds determined in the previous cell, this cell focuses on:
- Identifying the specific dates where the consumption was anomalously low or high for each meter.
- Displaying the anomaly dates, providing a direct look into days that might require further investigation.

In [ ]:
# For each meter, calculate Q1, Q3, IQR, and the bounds for outliers
anomaly_dates = {}

for column in all_consumption_columns:
    Q1 = daily_consumption[column].quantile(0.25)
    Q3 = daily_consumption[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - THRESHOLD_FOR_OUTLIERS * IQR
    upper_bound = Q3 + THRESHOLD_FOR_OUTLIERS * IQR
    
    # Extract dates of the anomalies
    anomaly_dates[column] = daily_consumption[(daily_consumption[column] < lower_bound) | (daily_consumption[column] > upper_bound)].index

# Display dates of anomalies for each meter
for meter, dates in anomaly_dates.items():
    print(f"Anomaly Dates for {meter}:")
    for date in dates:
        print(f"  - {date.strftime('%Y-%m-%d')}")
    print("\n")


##### 7.3 Visualizing Anomalies in Daily Energy Consumption**

Visualization aids in intuitively understanding the data. In this cell:
- We filter out the anomalies from the daily consumption data using previously set thresholds.
- Plot the daily energy consumption over time for each meter.
- Highlight the detected anomalies in red, differentiating them from the regular consumption patterns.

In [ ]:
# For each meter, calculate Q1, Q3, IQR, and the bounds for outliers
anomalies = {}

for column in all_consumption_columns:
    Q1 = daily_consumption[column].quantile(0.25)
    Q3 = daily_consumption[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - THRESHOLD_FOR_OUTLIERS * IQR
    upper_bound = Q3 + THRESHOLD_FOR_OUTLIERS * IQR
    
    # Filter out the anomalies
    anomalies[column] = daily_consumption[(daily_consumption[column] < lower_bound) | (daily_consumption[column] > upper_bound)]

# Plotting the daily consumption along with anomalies
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 15))

for idx, column in enumerate(all_consumption_columns):
    axes[idx].plot(daily_consumption.index, daily_consumption[column], label='Daily Consumption', color='blue')
    axes[idx].scatter(anomalies[column].index, anomalies[column][column], color='red', label='Anomalies')
    axes[idx].set_title(f'Daily Energy Consumption with Anomalies for {column}')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Daily Energy Consumption (kWh)')
    axes[idx].legend()

plt.tight_layout()
plt.show()

### Task 8: Solar Panel Production vs Consumption
One of the sustainable solutions to energy needs is harnessing solar power. In this section, we'll compare solar panel production against consumption. This will provide insights into how self-produced energy matches up against consumption and where additional energy sources might be needed.
Understanding the Problem:

Often, solar panels produce more energy than is consumed by a facility or household. This excess energy, if not stored or fed back into the grid, is essentially wasted. By identifying the days and the amount of excess energy produced, facilities can make informed decisions, such as investing in energy storage solutions or adjusting their energy consumption patterns.
Approach:

To visualize and calculate the excess energy produced by the solar panels, we take the following steps:

1. **Filtering Excess Intervals:**
For each 15-minute interval in the dataset, we identify periods where the energy generated by the solar panel exceeds the energy consumed.
    ```python
    excess_intervals = data[data[solar_panel_col] > data[meter_col]]
    ```

2. **Calculating Excess Energy:**
    For each of these intervals, we calculate the difference between the energy generated by the solar panel and the energy consumed. This difference represents the "excess energy" for that interval.
    ```python
    excess_intervals['Excess'] = excess_intervals[solar_panel_col] - excess_intervals[meter_col]
    ```

3. **Aggregating Excess Energy by Day:**
    We then aggregate this "excess energy" for each day to get a daily summary of excess energy production.
    ```python
    daily_excess = excess_intervals.groupby(excess_intervals['TimePeriodStart'].dt.date)['Excess'].sum()
    ```

4. **Visualization:**
    A bar chart provides a visual representation of the excess energy produced each day. This helps in quickly identifying days with higher excess energy production.
    ```python
    daily_excess.plot(kind='bar', color='skyblue')
    ```

#### 8.1 Comparing Sum of All Production vs. Sum of All Consumption
Step 1: Summation and Energy Balance Calculation
In this step, we'll sum up all the production columns (solar panels) and all the consumption columns (consumption meters + electric car chargers). We'll then determine the energy balance for each interval.

In [ ]:
# Summing up all consumption and production columns for the entire facility
processed_data['TotalFacilityConsumption'] = processed_data[all_consumption_columns].sum(axis=1)
processed_data['TotalFacilityProduction'] = processed_data[solar_panel_columns].sum(axis=1)

# Calculating Energy Balance for the entire facility: Positive if production > consumption, Negative otherwise
processed_data['FacilityEnergyBalance'] = processed_data['TotalFacilityProduction'] - processed_data['TotalFacilityConsumption']

# Aggregate this "net energy" for each day for the entire facility
daily_facility_data = processed_data.groupby(processed_data['TimePeriodStart'].dt.date).agg({'TotalFacilityConsumption': 'sum', 'TotalFacilityProduction': 'sum', 'FacilityEnergyBalance': 'sum'}).reset_index()

# Convert 'TimePeriodStart' back to datetime if it's not
daily_facility_data['TimePeriodStart'] = pd.to_datetime(daily_facility_data['TimePeriodStart'])

# Monthly aggregation for the entire facility
monthly_facility_data = daily_facility_data.groupby(daily_facility_data['TimePeriodStart'].dt.to_period("M")).agg({'TotalFacilityConsumption': 'sum', 'TotalFacilityProduction': 'sum', 'FacilityEnergyBalance': 'sum'}).reset_index()

# Creating a new DataFrame for the energy balances of the entire facility
facility_energy_balance_df = processed_data[['TimePeriodStart', 'TotalFacilityConsumption', 'TotalFacilityProduction', 'FacilityEnergyBalance']].copy()

##### 8.1.1 Print out monthly values for the sum of all production and the sum of all consumption

In [ ]:
# # Print out the monthly summaries for the entire facility
# for index, row in monthly_facility_data.iterrows():
#     month = row['TimePeriodStart'].strftime('%B %Y')
#     print(f"Month: {month}")
#     print(f"Monthly Consumption for Facility: {row['TotalFacilityConsumption']:.2f} kWh")
#     print(f"Monthly Production for Facility: {row['TotalFacilityProduction']:.2f} kWh")
#     print(f"Net Balance (Production - Consumption) for Facility: {row['FacilityEnergyBalance']:.2f} kWh")
#     print('---')

##### 8.1.2 Visual for the sum of all production and the sum of all consumption

In [ ]:
def plot_total_consumption_vs_production(data, all_consumption_columns, solar_panel_columns):
    # Use the pre-calculated total consumption and total production columns
    # Aggregate this "net energy" for each day
    daily_facility_data = data.groupby(data['TimePeriodStart'].dt.date).agg({'TotalFacilityConsumption': 'sum', 'TotalFacilityProduction': 'sum', 'FacilityEnergyBalance': 'sum'}).reset_index()
    
    # Plot settings
    plt.figure(figsize=(16, 8))
    
    # Set Seaborn style
    sns.set_style("whitegrid")
    
    # Plot area chart
    # Consumption
    plt.fill_between(daily_facility_data['TimePeriodStart'], daily_facility_data['TotalFacilityConsumption'], color="#1f77b4", label='Total Consumption', alpha=0.6)
    # Production
    plt.fill_between(daily_facility_data['TimePeriodStart'], 0, daily_facility_data['TotalFacilityProduction'], color="#ff7f0e", label='Total Production', alpha=0.6)
    # Net (Excess/Deficit)
    plt.fill_between(daily_facility_data['TimePeriodStart'], 0, daily_facility_data['FacilityEnergyBalance'], color="#2ca02c", label='Net (Excess/Deficit)', alpha=0.6, where=(daily_facility_data['FacilityEnergyBalance'] > 0))
    plt.fill_between(daily_facility_data['TimePeriodStart'], 0, daily_facility_data['FacilityEnergyBalance'], color="#d62728", label='Net (Deficit)', alpha=0.6, where=(daily_facility_data['FacilityEnergyBalance'] <= 0))

    plt.title('Daily Total Consumption vs. Total Production for Facility')
    plt.ylabel('Energy (kWh)')
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Call the function
plot_total_consumption_vs_production(processed_data, all_consumption_columns, solar_panel_columns)


##### 8.1.3 Inspection of the total energy balance

In [ ]:
# 15-minute interval data
# du.inspect_dataframe(facility_energy_balance_df)

In [ ]:
# Daily data
# du.inspect_dataframe(daily_facility_data)

In [ ]:
# Monthly data
# du.inspect_dataframe(monthly_facility_data)

##### 8.1.4 Export dataframe into excel file

In [ ]:
filename = "facility_total_energy_balance"
# du.export_dataframe_to_excel(facility_energy_balance_df, filename, EXPORT_FILE_PATH)

In [ ]:
filename = "facility_total_daily_energy_balance"
# du.export_dataframe_to_excel(facility_energy_balance_df, filename, EXPORT_FILE_PATH)

In [ ]:
filename = "facility_total_monthly_energy_balance"
# du.export_dataframe_to_excel(facility_energy_balance_df, filename, EXPORT_FILE_PATH)

#### 8.2 Comparing the consumption and production by pods
Facilities may consist of multiple "pods", which are sub-units or sections. Each pod can have its own consumption meter and solar panel. Understanding the energy dynamics at the pod level can help in identifying the consumption patterns of different sections of a facility and how well the solar production of each pod matches its consumption.

##### 8.2.1 Main function for the consumption and production by pods

In [ ]:
def plot_and_calculate_net_area_chart(data, meter_col, solar_panel_col):
    # Calculate the net energy for each interval (positive or negative)
    data['Net'] = data[solar_panel_col] - data[meter_col]

    # Aggregate this "net energy" for each day
    daily_data = data.groupby(data['TimePeriodStart'].dt.date).agg({meter_col: 'sum', solar_panel_col: 'sum', 'Net': 'sum'}).reset_index()

    # Monthly aggregation
    monthly_data = data.groupby(data['TimePeriodStart'].dt.to_period("M")).agg({meter_col: 'sum', solar_panel_col: 'sum', 'Net': 'sum'}).reset_index()

    # Plot settings
    plt.figure(figsize=(16, 8))
    
    # Set Seaborn style
    sns.set_style("whitegrid")
    
    # Plot area chart
    # Consumption
    plt.fill_between(daily_data['TimePeriodStart'], daily_data[meter_col], color="#1f77b4", label='Consumption', alpha=0.6)
    # Production
    plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data[solar_panel_col], color="#ff7f0e", label='Production', alpha=0.6)
    # Net (Excess/Deficit)
    plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data['Net'], color="#2ca02c", label='Net (Excess/Deficit)', alpha=0.6, where=(daily_data['Net'] > 0))
    plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data['Net'], color="#d62728", label='Net (Deficit)', alpha=0.6, where=(daily_data['Net'] <= 0))

    plt.title(f'Daily Consumption, Production, and Net Solar Production for {meter_col}')
    plt.ylabel('Energy (kWh)')
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

##### 8.2.2 The loop for the consumption and production by pods

In [ ]:
# Create empty dictionaries to store aggregated data for each pod
daily_pod_data_dict = {}
monthly_pod_data_dict = {}
pod_energy_balance_df_dict = {}

# Loop through each pod's meter and solar panel columns
for meter, solar_panel in pods.items():
    if solar_panel[1] in processed_data.columns:
        
        # Summing up consumption and production columns for each pod
        processed_data[f'{meter}Consumption'] = processed_data[meter]
        processed_data[f'{meter}Production'] = processed_data[solar_panel[1]]
        
        # Calculating Energy Balance for each pod: Positive if production > consumption, Negative otherwise
        processed_data[f'{meter}EnergyBalance'] = processed_data[f'{meter}Production'] - processed_data[f'{meter}Consumption']
        
        # Aggregate this "net energy" for each day for each pod
        daily_pod_data = processed_data.groupby(processed_data['TimePeriodStart'].dt.date).agg({f'{meter}Consumption': 'sum', f'{meter}Production': 'sum', f'{meter}EnergyBalance': 'sum'}).reset_index()
        daily_pod_data['TimePeriodStart'] = pd.to_datetime(daily_pod_data['TimePeriodStart'])
        
        # Monthly aggregation for each pod
        monthly_pod_data = daily_pod_data.groupby(daily_pod_data['TimePeriodStart'].dt.to_period("M")).agg({f'{meter}Consumption': 'sum', f'{meter}Production': 'sum', f'{meter}EnergyBalance': 'sum'}).reset_index()
        
        # Store the aggregated data in the dictionaries
        daily_pod_data_dict[meter] = daily_pod_data
        monthly_pod_data_dict[meter] = monthly_pod_data
        
        # Creating a new DataFrame for the energy balances of each pod
        pod_energy_balance_df_dict[meter] = processed_data[['TimePeriodStart', f'{meter}Consumption', f'{meter}Production', f'{meter}EnergyBalance']].copy()
        
        # Call the function to plot data for each pod
        plot_and_calculate_net_area_chart(processed_data, meter, solar_panel[1])

##### 8.2.3 Inspect and export of the total energy balance by pods

In [ ]:
# Inspect and export dataframes for each pod
for pod_meter, solar_panel in pods.items():
    if solar_panel[1] in processed_data.columns:
        
        # 15-minute interval data
        print(f"15-minute interval data for {pod_meter}:")
        # du.inspect_dataframe(pod_energy_balance_df_dict[pod_meter])
        
        # Daily data
        print(f"Daily data for {pod_meter}:")
        # du.inspect_dataframe(daily_pod_data_dict[pod_meter])
        
        # Monthly data
        print(f"Monthly data for {pod_meter}:")
        # du.inspect_dataframe(monthly_pod_data_dict[pod_meter])
        
        # Export 15-minute interval data
        filename = f"{pod_meter}_energy_balance"
        # du.export_dataframe_to_excel(pod_energy_balance_df_dict[pod_meter], filename, EXPORT_FILE_PATH)
        
        # Export daily data
        filename = f"{pod_meter}_daily_energy_balance"
        # du.export_dataframe_to_excel(daily_pod_data_dict[pod_meter], filename, EXPORT_FILE_PATH)
        
        # Export monthly data
        filename = f"{pod_meter}_monthly_energy_balance"
        # du.export_dataframe_to_excel(monthly_pod_data_dict[pod_meter], filename, EXPORT_FILE_PATH)

##### OLD VISUALS (TO BE DELETED)

In [ ]:
# def plot_and_calculate_aggregated_net_area_chart(data, all_consumption_columns, solar_panel_columns):
#     # Calculate the aggregated consumption and production for each interval
#     data['Total_Consumption'] = data[all_consumption_columns].sum(axis=1)
#     data['Total_Production'] = data[solar_panel_columns].sum(axis=1)
    
#     # Calculate the net energy for each interval (positive or negative)
#     data['Net'] = data['Total_Production'] - data['Total_Consumption']

#     # Aggregate these values for each day
#     daily_data = data.groupby(data['TimePeriodStart'].dt.date).agg({'Total_Consumption': 'sum', 'Total_Production': 'sum', 'Net': 'sum'}).reset_index()

#     # Plot settings
#     plt.figure(figsize=(16, 8))
    
#     # Set Seaborn style
#     sns.set_style("whitegrid")
    
#     # Plot area chart
#     # Consumption
#     plt.fill_between(daily_data['TimePeriodStart'], daily_data['Total_Consumption'], color="#1f77b4", label='Consumption', alpha=0.6)
#     # Production
#     plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data['Total_Production'], color="#ff7f0e", label='Production', alpha=0.6)
#     # Net (Excess/Deficit)
#     plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data['Net'], color="#2ca02c", label='Net (Excess/Deficit)', alpha=0.6, where=(daily_data['Net'] > 0))
#     plt.fill_between(daily_data['TimePeriodStart'], 0, daily_data['Net'], color="#d62728", label='Net (Deficit)', alpha=0.6, where=(daily_data['Net'] <= 0))

#     plt.title('Total Daily Aggregated Consumption, Production, and Net Solar Production')
#     plt.ylabel('Energy (kWh)')
#     plt.xlabel('Date')
#     plt.legend(loc='upper left')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

# # Call the function
# plot_and_calculate_aggregated_net_area_chart(processed_data, all_consumption_columns, solar_panel_columns)

In [ ]:
# def plot_and_calculate_net_seaborn(data, meter_col, solar_panel_col):
#     # Calculate the net energy for each interval (positive or negative)
#     data['Net'] = data[solar_panel_col] - data[meter_col]

#     # Aggregate this "net energy" for each day
#     daily_data = data.groupby(data['TimePeriodStart'].dt.date).agg({meter_col: 'sum', solar_panel_col: 'sum', 'Net': 'sum'}).reset_index()

#     # Aggregate the monthly consumption, production, and net balance
#     monthly_data = data.groupby(data['TimePeriodStart'].dt.to_period("M")).agg({meter_col: 'sum', solar_panel_col: 'sum', 'Net': 'sum'}).reset_index()

#     # Print out the monthly summaries
#     for index, row in monthly_data.iterrows():
#         month = row['TimePeriodStart'].strftime('%B %Y')
#         print(f"Month: {month}")
#         print(f"Monthly Consumption for {meter_col}: {row[meter_col]:.2f} kWh")
#         print(f"Monthly Production from {solar_panel_col}: {row[solar_panel_col]:.2f} kWh")
#         print(f"Net Balance (Production - Consumption) for {meter_col}: {row['Net']:.2f} kWh")
#         print('---')

#     # Seaborn Lineplot Visualization
#     plt.figure(figsize=(16, 8))
#     sns.lineplot(data=daily_data, x='TimePeriodStart', y=meter_col, label='Consumption', color='#1f77b4')  # Blue
#     sns.lineplot(data=daily_data, x='TimePeriodStart', y=solar_panel_col, label='Production', color='#ff7f0e')  # Orange
#     sns.lineplot(data=daily_data, x='TimePeriodStart', y='Net', label='Net (Excess/Deficit)', color='skyblue')
    
#     plt.title(f'Daily Consumption and Net Solar Production for {meter_col}')
#     plt.ylabel('Energy (kWh)')
#     plt.xlabel('Date')
#     plt.legend(loc='upper left')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

#     # Print out total net energy
#     total_net_energy = daily_data['Net'].sum()
#     print(f"Total Net Energy for {meter_col}: {total_net_energy:.2f} kWh")

# # Ensure the solar panel column exists in the data
# for meter, solar_panel in pods.items():
#     if solar_panel[1] in processed_data.columns:
#         plot_and_calculate_net_seaborn(processed_data, meter, solar_panel[1])

#### 8.3 Monthly Stacked Area Charts
The goal of this section is to visualize the monthly energy consumption and solar production using stacked area charts. This visual representation provides insights into the magnitude of energy production and consumption over the months, as well as indicating periods of energy surplus or deficit.

##### 8.3.1 Monthly Stacked Area Chart for Total Facility
The first chart will represent the energy dynamics for the entire facility. It will illustrate the total energy consumption and production across all pods and meters.

In [ ]:
def plot_monthly_stacked_area_chart_total(data, consumption_columns, production_columns):
    # Group data by month and sum values
    monthly_data = data.groupby(data['TimePeriodStart'].dt.to_period("M"))[consumption_columns + production_columns].sum().reset_index()
    monthly_data['TimePeriodStart'] = monthly_data['TimePeriodStart'].dt.to_timestamp()
    
    monthly_data['TotalConsumption'] = monthly_data[consumption_columns].sum(axis=1)
    monthly_data['TotalProduction'] = monthly_data[production_columns].sum(axis=1)

    # Calculate the monthly excess energy
    monthly_data['Excess'] = monthly_data['TotalProduction'] - monthly_data['TotalConsumption']
    monthly_excess = monthly_data[['TimePeriodStart', 'Excess']]

    plt.figure(figsize=(16, 8))
    plt.stackplot(monthly_data['TimePeriodStart'], monthly_data['TotalConsumption'], monthly_data['TotalProduction'], labels=['Total Consumption', 'Total Production'], alpha=0.6)
    plt.title('Monthly Total Energy Consumption vs Total Solar Production for Facility')
    plt.ylabel('Energy (kWh)')
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print out monthly excess or deficit energy
    for _, row in monthly_excess.iterrows():
        if row['Excess'] < 0:
            print(f"Total Energy Deficit for {row['TimePeriodStart'].strftime('%B %Y')}: {-row['Excess']:.2f} kWh")
        else:
            print(f"Total Excess Energy for {row['TimePeriodStart'].strftime('%B %Y')}: {row['Excess']:.2f} kWh")

plot_monthly_stacked_area_chart_total(processed_data, all_consumption_columns, solar_panel_columns)

##### 8.3.2 Monthly Stacked Area Charts by Pods
Now, we'll break down the energy dynamics for each individual pod. Each chart will illustrate the energy consumption and production for a specific pod.

In [ ]:
def plot_monthly_stacked_area_chart_pod(data, meter_col, solar_panel_col):
    # Calculate the net energy for each interval (positive or negative)
    data['Net'] = data[solar_panel_col] - data[meter_col]

    # Aggregate this "net energy" for each month
    monthly_data = data.groupby(data['TimePeriodStart'].dt.to_period("M")).agg({meter_col: 'sum', solar_panel_col: 'sum', 'Net': 'sum'}).reset_index()
    monthly_data['TimePeriodStart'] = monthly_data['TimePeriodStart'].dt.to_timestamp()

    # Calculate the monthly excess energy
    monthly_data['Excess'] = monthly_data[solar_panel_col] - monthly_data[meter_col]
    monthly_excess = monthly_data[['TimePeriodStart', 'Excess']]

    plt.figure(figsize=(16, 8))
    plt.stackplot(monthly_data['TimePeriodStart'], monthly_data[meter_col], monthly_data[solar_panel_col], labels=['Consumption', 'Production'], alpha=0.6)
    plt.title(f'Monthly Energy Consumption vs Solar Production for {meter_col}')
    plt.ylabel('Energy (kWh)')
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Print out monthly excess or deficit energy
    for _, row in monthly_excess.iterrows():
        if row['Excess'] < 0:
            print(f"Energy Deficit for {row['TimePeriodStart'].strftime('%B %Y')} from {meter_col}: {-row['Excess']:.2f} kWh")
        else:
            print(f"Excess Energy for {row['TimePeriodStart'].strftime('%B %Y')} from {meter_col}: {row['Excess']:.2f} kWh")


In [ ]:
for meter, solar_panel in pods.items():
    if solar_panel[1] in processed_data.columns:  # Ensure the solar panel column exists
        plot_monthly_stacked_area_chart_pod(processed_data, meter, solar_panel[1])

### Task 9: Understanding Car Charger Energy Consumption
In this task, we'll adjust the consumption patterns of the electric car charging circuit throughout the day. We aim to optimize car charging habits to maximize saved energy. By redistributing the charging consumption, we can ensure that energy is used more efficiently, especially during peak solar production hours.

***Steps Involved:***
1. Adjust the Charging Circuit Consumption:
    - Adjust the consumption values of the electric car charging circuit based on specific thresholds throughout the day.

2. Aggregate Total Consumption:
    - Compute the total consumption across all meters, including electric car chargers, for every 15-minute interval.

3. Smooth Out the Consumption Curve:
    - Distribute the consumption of the charger evenly between peak solar hours to ensure a consistent power load.

4. Visualization and Calculation:
    - Compare the original and adjusted total consumption visually and quantify the energy savings resulting from the adjustments.


#### 9.1 Logic and calculations for the car charger energy consumption

##### 9.1.1 Logic for the car charger energy consumption

1. ***Step: Duplicate the Original Dataframe***
    - Before making any modifications, create a copy of the original data. This step ensures that all subsequent changes are made to this duplicated dataset, preserving the original data intact.

2. ***Step: Adjust Charging Circuit Consumption Based on Time of Day***
    - Using specific time intervals, set upper limits on the energy consumption of the electric car charging circuit.
    - Between midnight and 10 AM, the consumption is capped at 3 kWh every 15 minutes.
    - Between 10 AM and 3 PM, the consumption is capped at 25 kWh every 15 minutes.

3. ***Step: Aggregate the Total Consumption:***
    - Calculate the total energy consumption across all meters (including the electric car chargers) for every row (15-minute interval) in the dataset.

4. ***Step: Smooth Out Midday Consumption:***
    - Evenly distribute the consumption of the car charger during midday hours to create a more consistent load profile.
    - Calculate the total consumption of the car charger during these hours.
    - Compute the average consumption by dividing this total by the number of 15-minute intervals in the midday period.
    - Distribute this average consumption evenly across all midday intervals.
    - Update the dataset with this adjusted consumption profile and recompute the total adjusted consumption.

5. Step: Visualization and Calculation:
    - Plot the original and adjusted average consumption for each 15-minute interval throughout the day.
    - The goal is to visually compare the original and adjusted consumption profiles and identify periods of increased or reduced energy usage.
    - The area between the two curves is shaded to highlight the difference in consumption due to the adjustments.

##### 9.1.2 Calculations and logic for the car charger energy consumption

In [ ]:
# Step 0: Adjust the Charging Circuit Consumption
adjusted_data = processed_data.copy()

# Step 1: Adjust the Charging Circuit Consumption
adjusted_data = processed_data.copy()

# Between 00:00 and 10:00, cap the consumption to 3kWh/15min.
mask_morning = (adjusted_data['Hour'] < 10)
adjusted_data.loc[mask_morning, electric_car_charger_columns] = adjusted_data.loc[mask_morning, electric_car_charger_columns].clip(upper=3)

# Between 10:00 and 15:00, cap the consumption to 25kWh/15min.
mask_midday = (adjusted_data['Hour'] >= 10) & (adjusted_data['Hour'] < 15)
adjusted_data.loc[mask_midday, electric_car_charger_columns] = adjusted_data.loc[mask_midday, electric_car_charger_columns].clip(upper=25)

# Step 2: Aggregate Total Consumption
adjusted_data['TotalConsumption'] = adjusted_data[all_consumption_columns].sum(axis=1)

# Step 3: Smooth Out the Consumption Curve
total_charge_midday = adjusted_data.loc[mask_midday, electric_car_charger_columns].sum()
num_intervals_midday = adjusted_data[mask_midday].shape[0]
even_charge = total_charge_midday / num_intervals_midday

adjusted_data.loc[mask_midday, electric_car_charger_columns] = even_charge
adjusted_data['AdjustedTotalConsumption'] = adjusted_data[all_consumption_columns].sum(axis=1)

# Calculate the total energy saved or lost after adjustments
total_original_consumption = adjusted_data['TotalConsumption'].sum()
total_adjusted_consumption = adjusted_data['AdjustedTotalConsumption'].sum()
energy_difference = total_original_consumption - total_adjusted_consumption

# Extract month from the 'TimePeriodStart' column
adjusted_data['MonthYear'] = adjusted_data['TimePeriodStart'].dt.to_period('M')

##### 9.1.3 Visual for the car charger energy consumption

In [ ]:
# Step 4: Visualization and Calculation
avg_original = adjusted_data.groupby(['Hour', 'Minute'])['TotalConsumption'].mean().reset_index()
avg_adjusted = adjusted_data.groupby(['Hour', 'Minute'])['AdjustedTotalConsumption'].mean().reset_index()

intra_day_comparison = pd.merge(avg_original, avg_adjusted, on=['Hour', 'Minute'], how='inner')
intra_day_comparison['Time'] = intra_day_comparison['Hour'].astype(str).str.zfill(2) + ':' + intra_day_comparison['Minute'].astype(str).str.zfill(2)
intra_day_comparison.rename(columns={"TotalConsumption": "Original Average Consumption", "AdjustedTotalConsumption": "Adjusted Average Consumption"}, inplace=True)

plt.figure(figsize=(16, 8))

sns.lineplot(x='Time', y='Original Average Consumption', data=intra_day_comparison, label='Original Average Consumption', linewidth=2)
sns.lineplot(x='Time', y='Adjusted Average Consumption', data=intra_day_comparison, label='Adjusted Average Consumption', linewidth=2)

plt.fill_between(intra_day_comparison['Time'], 
                 intra_day_comparison['Original Average Consumption'], 
                 intra_day_comparison['Adjusted Average Consumption'], 
                 where=(intra_day_comparison['Adjusted Average Consumption'] < intra_day_comparison['Original Average Consumption']),
                 color='red', alpha=0.4, label='Reduced Consumption')
plt.fill_between(intra_day_comparison['Time'], 
                 intra_day_comparison['Original Average Consumption'], 
                 intra_day_comparison['Adjusted Average Consumption'], 
                 where=(intra_day_comparison['Adjusted Average Consumption'] > intra_day_comparison['Original Average Consumption']),
                 color='green', alpha=0.4, label='Increased Consumption')

plt.xlabel('Time of Day')
plt.ylabel('Average Consumption (kWh)')
plt.title('Intra-day Comparison of Original and Adjusted Average Consumption')
plt.xticks(rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

##### 9.1.4 Monthly breakdown of savings of the adjusted consumption

In [ ]:
# Group by month and compute the sum of original and adjusted consumption for each month
monthly_sums = adjusted_data.groupby('MonthYear').agg({
    'TotalConsumption': 'sum',
    'AdjustedTotalConsumption': 'sum'
}).reset_index()

# Compute the monthly energy difference
monthly_sums['MonthlyEnergyDifference'] = monthly_sums['TotalConsumption'] - monthly_sums['AdjustedTotalConsumption']

# Print the saved energy by month
for _, row in monthly_sums.iterrows():
    month = row['MonthYear'].strftime('%B %Y')
    difference = row['MonthlyEnergyDifference']
    if difference > 0:
        print(f"Energy saved in {month}: {difference:.2f} kWh")
    else:
        print(f"Energy increased in {month}: {-difference:.2f} kWh")

### Task 10: Final Observations and Conclusions

After analyzing various facets of the energy consumption data, it's essential to consolidate our findings, draw conclusions, and provide actionable insights. In this section, we'll summarize the key observations and suggest further steps or considerations.

#TODO: Add final thoughts